In [1]:

import os
#os.chdir("/kaggle/input/trainfoodjson")
!ls
!pwd


__notebook_source__.ipynb
/kaggle/working


In [2]:

!pip install transformers
!pip install fastBPE
!pip install fairseq
!pip install vncorenlp

  Preparing metadata (setup.py) ... done
  Created wheel for fastBPE: filename=fastBPE-0.1.0-cp37-cp37m-linux_x86_64.whl size=746411 sha256=9b7fce002e19265063b5db21680364d6201e0485f5299e193dd8b0b51fcd2f8e
  Stored in directory: /root/.cache/pip/wheels/bd/d4/0e/0d317a65f77d3f8049fedd8a2ee0519164cf3e6bd77ef886f1
Successfully built fastBPE
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.0/11.0 MB 31.0 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.4/235.4 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141230 sha256=e043279ae32d9b2442bd34f487b31d7b57d473b21e4774417b6cd91581905bd5
  Stored in directory: /r

In [3]:
!ls

__notebook_source__.ipynb


In [4]:
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
import argparse
from tqdm import tqdm
import json
from vncorenlp import VnCoreNLP
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
from transformers import RobertaForSequenceClassification, RobertaConfig, AdamW
import random
from tqdm import tqdm_notebook
from sklearn.metrics import roc_auc_score
import pandas as pd


#rdrsegmenter = VnCoreNLP("./vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 


In [ ]:
!ls

**Deploy PhoBert**

In [5]:
# Install the vncorenlp python wrapper
# !pip install vncorenlp

# Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter) 
!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/
!ls
rdrsegmenter = VnCoreNLP("./vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 


--2022-12-12 14:44:03--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.1.1.jar’

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M  --.-KB/s    in 0.1s    

2022-12-12 14:44:03 (220 MB/s) - ‘VnCoreNLP-1.1.1.jar’ saved [27412575/27412575]

--2022-12-12 14:44:04--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting respon

In [6]:
text = "Đại học Quốc gia Hà Nội."
word_segmented_text = rdrsegmenter.tokenize(text) 
print(word_segmented_text)

[['Đại_học', 'Quốc_gia', 'Hà_Nội', '.']]


In [7]:
!wget https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
!tar -xzvf PhoBERT_base_fairseq.tar.gz
!wget https://public.vinai.io/PhoBERT_base_transformers.tar.gz
!tar -xzvf PhoBERT_base_transformers.tar.gz


--2022-12-12 14:44:38--  https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
Resolving public.vinai.io (public.vinai.io)... 54.230.31.74, 54.230.31.76, 54.230.31.108, ...
Connecting to public.vinai.io (public.vinai.io)|54.230.31.74|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1243308020 (1.2G) [application/x-tar]
Saving to: ‘PhoBERT_base_fairseq.tar.gz’

PhoBERT_base_fairse 100%[===================>]   1.16G  52.5MB/s    in 23s     

2022-12-12 14:45:01 (51.2 MB/s) - ‘PhoBERT_base_fairseq.tar.gz’ saved [1243308020/1243308020]

PhoBERT_base_fairseq/
PhoBERT_base_fairseq/bpe.codes
PhoBERT_base_fairseq/model.pt
PhoBERT_base_fairseq/dict.txt
--2022-12-12 14:45:17--  https://public.vinai.io/PhoBERT_base_transformers.tar.gz
Resolving public.vinai.io (public.vinai.io)... 54.230.31.104, 54.230.31.76, 54.230.31.74, ...
Connecting to public.vinai.io (public.vinai.io)|54.230.31.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 322405979 (30

In [8]:
!ls

PhoBERT_base_fairseq	     PhoBERT_base_transformers.tar.gz
PhoBERT_base_fairseq.tar.gz  __notebook_source__.ipynb
PhoBERT_base_transformers    vncorenlp


In [9]:
#Build Train
MAX_LEN = 256
parser = argparse.ArgumentParser()
parser.add_argument('--bpe-codes', 
    default="./PhoBERT_base_transformers/bpe.codes",
    required=False,
    type=str,
    help='path to fastBPE BPE'
)
args, unknown = parser.parse_known_args()
bpe = fastBPE(args)

# Load the dictionary
vocab = Dictionary()
vocab.add_from_file("./PhoBERT_base_transformers/dict.txt")

train_path = '/kaggle/input/trainfoodjson/train.json'
full_dataset = []

with open(train_path, 'r') as f:
    datasets = json.load(f)
# 

for sample in tqdm(datasets):
    label = int(sample['Rating'] >= 6)
    comment = str(sample['Comment'])
    comment = rdrsegmenter.tokenize(comment)
    comment = ' '.join([' '.join(x) for x in comment])
    subwords = '<s> ' + bpe.encode(comment) + ' </s>'
    encoded_comment = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
    ids = pad_sequences([encoded_comment], maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
    ids = ids[0]
    mask = [int(token_id > 0) for token_id in ids]
    full_dataset.append([ids, mask, label])
    # if cnt == 15:
    #     break
    # cnt += 1
full_dataset = np.array(full_dataset)
kf = KFold(n_splits=10, shuffle=True)
print()

partition = 1
for tid, vid in kf.split(full_dataset):
    print(f"Partition {partition}")
    train_set = full_dataset[tid]
    train_positive = sum(pos[2] for pos in train_set)
    val_set = full_dataset[vid]
    val_positive = sum(pos[2] for pos in val_set)
    print(f"Number of positive label in train data: {train_positive}/{len(train_set)}")
    print(f"Number of positive label in train data: {val_positive}/{len(val_set)}")
    partition += 1
    print("#")

Loading codes from ./PhoBERT_base_transformers/bpe.codes ...
Read 64000 codes from the codes file.
100%|██████████| 9071/9071 [00:57<00:00, 157.83it/s]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



Partition 1
Number of positive label in train data: 6423/8163
Number of positive label in train data: 724/908
#
Partition 2
Number of positive label in train data: 6433/8164
Number of positive label in train data: 714/907
#
Partition 3
Number of positive label in train data: 6450/8164
Number of positive label in train data: 697/907
#
Partition 4
Number of positive label in train data: 6430/8164
Number of positive label in train data: 717/907
#
Partition 5
Number of positive label in train data: 6453/8164
Number of positive label in train data: 694/907
#
Partition 6
Number of positive label in train data: 6407/8164
Number of positive label in train data: 740/907
#
Partition 7
Number of positive label in train data: 6429/8164
Number of positive label in train data: 718/907
#
Partition 8
Number of positive label in train data: 6424/8164
Number of positive label in train data: 723/907
#
Partition 9
Number of positive label in train data: 6441/8164
Number of positive label in train data: 7

In [12]:
#Build Loader
def build_loader(dataset, train_id, val_id):
    train_set = dataset[train_id]
    val_set = dataset[val_id]
    train_inputs = torch.tensor([s[0] for s in train_set])
    val_inputs = torch.tensor([s[0] for s in val_set])
    train_targets = torch.tensor([s[2] for s in train_set])
    val_targets = torch.tensor([s[2] for s in val_set])
    train_masks = torch.tensor([s[1] for s in train_set])
    val_masks = torch.tensor([s[1] for s in val_set])

    train_data = TensorDataset(train_inputs, train_masks, train_targets)
    train_sampler = SequentialSampler(train_data)
    train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)

    val_data = TensorDataset(val_inputs, val_masks, val_targets)
    val_sampler = SequentialSampler(val_data)
    val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=32)
    return train_dataloader, val_dataloader

In [11]:
#Build Test
df = pd.read_csv("/kaggle/input/dataset/test.csv")
test_datasets = []
for _, row in df[["RevId", "Comment"]].iterrows():
    test_datasets.append([row["RevId"], row["Comment"]])
test_set = []
for sample in tqdm(test_datasets):
    comment = str(sample[1])
    comment = rdrsegmenter.tokenize(comment)
    comment = ' '.join([' '.join(x) for x in comment])
    subwords = '<s> ' + bpe.encode(comment) + ' </s>'
    encoded_comment = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
    ids = pad_sequences([encoded_comment], maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
    ids = ids[0]
    mask = [int(token_id > 0) for token_id in ids]
    test_set.append([sample[0], ids, mask])
test_revid = torch.tensor([s[0] for s in test_set])
test_inputs = torch.tensor([s[1] for s in test_set])
test_masks = torch.tensor([s[2] for s in test_set])

test_data = TensorDataset(test_revid, test_inputs, test_masks)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=32)

100%|██████████| 5103/5103 [00:28<00:00, 182.25it/s]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:207.)


**Load Model**

In [13]:
config = RobertaConfig.from_pretrained(
    "./PhoBERT_base_transformers/config.json", from_tf=False, num_labels = 1, output_hidden_states=False,
)
BERT_SA = RobertaForSequenceClassification.from_pretrained(
    "./PhoBERT_base_transformers/model.bin",
    config=config
)

You are using a model of type bert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at ./PhoBERT_base_transformers/model.bin were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenc

In [14]:
import torch
import torch.nn.functional as F


def sigmoid_focal_loss(
    inputs: torch.Tensor,
    targets: torch.Tensor,
    alpha: float = 0.25,
    gamma: float = 2,
    reduction: str = "mean",
):
    p = torch.sigmoid(inputs)
    ce_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction="none")
    p_t = p * targets + (1 - p) * (1 - targets)
    loss = ce_loss * ((1 - p_t) ** gamma)

    if alpha >= 0:
        alpha_t = alpha * targets + (1 - alpha) * (1 - targets)
        loss = alpha_t * loss

    if reduction == "mean":
        loss = loss.mean()
    elif reduction == "sum":
        loss = loss.sum()

    return loss

def np_sigmoid(x):
    return 1/(1 + np.exp(-x))

In [15]:
def save_model(model, optimizer, path):
    state_dict = {
        "optimizer": optimizer.state_dict(),
        "model": model.state_dict()
    }
    torch.save(state_dict, path)

**TRAINING**

In [19]:
device = 'cuda'
epochs = 10
BERT_SA.to(device)
print('Done')
param_optimizer = list(BERT_SA.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=1e-5, correct_bias=False)


for epoch_i, (tid, vid) in enumerate(kf.split(full_dataset)):
    print('======== Partition {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    total_loss = 0
    BERT_SA.train()
    train_logits, train_lbls = 0, 0
    nb_train_steps = 0
    train_f1 = 0
    train_dataloader, val_dataloader = build_loader(full_dataset, tid, vid)
    for step, batch in tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc="Training:"):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2]
        b_labels = b_labels.to(device)

        BERT_SA.zero_grad()
        outputs = BERT_SA(b_input_ids, 
            token_type_ids=None, 
            attention_mask=b_input_mask, 
            labels=b_labels)

        loss = sigmoid_focal_loss(outputs[1].squeeze(1), b_labels.float())
        total_loss += loss.item()
        
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # tmp_train_roc_auc_score = flat_roc_auc_score(label_ids, logits)
        if step == 0:
            train_logits = logits.flatten()
            train_lbls = label_ids.flatten()
        else:
            train_logits = np.concatenate([train_logits, logits.flatten()])
            train_lbls = np.concatenate([train_lbls, label_ids.flatten()])
        nb_train_steps += 1
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(BERT_SA.parameters(), 1.0)
        optimizer.step()
        
    avg_train_loss = total_loss / len(train_dataloader)
    print("ROC-AUC Score: {0:.4f}".format(roc_auc_score(train_lbls, train_logits, average='macro')))
    print("Average training loss: {0:.4f}".format(avg_train_loss))
    print("#")
    print("Running Validation...")
    BERT_SA.eval()
    eval_loss, eval_logits, eval_lbls = 0, 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    for step, batch in tqdm(enumerate(val_dataloader), total=len(val_dataloader), desc="Validating:"):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
            outputs = BERT_SA(b_input_ids, 
            token_type_ids=None, 
            attention_mask=b_input_mask)
            logits = outputs[0]
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            if step == 0:
                eval_logits = logits.flatten()
                eval_lbls = label_ids.flatten()
            else:
                eval_logits = np.concatenate([eval_logits, logits.flatten()])
                eval_lbls = np.concatenate([eval_lbls, label_ids.flatten()])
            nb_eval_steps += 1
    print(eval_logits.shape)
    print([round(float(rate) * 10, 1) for rate in np_sigmoid(eval_logits)])
    print("ROC-AUC score: {0:.4f}".format(roc_auc_score(eval_lbls, eval_logits, average='macro')))
    print("#")
    print("Running Test...")
    test_revid, test_logits = 0, 0
    BERT_SA.eval()
    for step, batch in tqdm(enumerate(test_dataloader), total=len(test_dataloader), desc="Testing..."):
        b_revids = batch[0]
        b_input_ids = batch[1].to(device)
        b_input_mask = batch[2].to(device)
       
        with torch.no_grad():
            outputs = BERT_SA(b_input_ids, 
            token_type_ids=None, 
            attention_mask=b_input_mask)
            logits = outputs[0]
            logits = logits.detach().cpu().numpy()
            if step == 0:
                test_logits = logits.flatten()
                test_revids = b_revids.flatten()
            else:
                test_logits = np.concatenate([test_logits, logits.flatten()])
                test_revids = np.concatenate([test_revids, b_revids.flatten()])
        results = {
            "RevId": [int(id) for id in test_revids],
            "Rating": [round(float(rate)*10, 1) for rate in np_sigmoid(test_logits)]
        }
        df = pd.DataFrame(results)
        df.to_csv(f"/kaggle/working/submit_{epoch_i}.csv")
    save_model(BERT_SA, optimizer, f"/kaggle/working/state_dict_{epoch_i}.pth")
print("Training complete!")

Done
======== Partition 1 / 10 ========
Training...


/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
Training:: 100%|██████████| 256/256 [03:09<00:00,  1.35it/s]


ROC-AUC Score: 0.9706
Average training loss: 0.0191
#
Running Validation...


Validating:: 100%|██████████| 29/29 [00:06<00:00,  4.28it/s]


(908,)
[1.6, 8.3, 8.1, 7.9, 8.1, 7.4, 8.0, 8.2, 7.8, 8.0, 7.9, 7.9, 8.0, 1.0, 7.6, 8.3, 7.4, 8.4, 7.8, 8.1, 8.3, 7.9, 8.1, 8.0, 1.1, 7.6, 8.5, 2.8, 8.3, 8.1, 7.7, 1.8, 8.3, 8.0, 1.1, 8.1, 8.3, 1.2, 5.8, 6.1, 1.4, 7.2, 7.8, 1.5, 6.1, 8.5, 6.3, 7.9, 7.3, 8.0, 7.5, 7.5, 1.0, 8.1, 7.3, 1.0, 8.3, 1.0, 3.5, 1.8, 7.5, 1.1, 8.1, 7.9, 8.1, 8.3, 8.1, 8.0, 7.7, 7.8, 8.7, 8.1, 7.7, 1.9, 8.3, 6.8, 1.1, 7.6, 8.2, 7.8, 1.9, 8.4, 8.0, 7.8, 6.8, 8.5, 7.0, 7.6, 7.9, 7.4, 8.3, 6.1, 6.4, 7.5, 7.7, 7.1, 8.1, 8.4, 7.8, 8.3, 3.1, 5.7, 1.0, 8.2, 7.5, 5.6, 1.5, 8.6, 8.4, 6.5, 7.8, 7.9, 8.4, 1.6, 8.0, 7.9, 7.1, 7.9, 8.2, 8.4, 8.0, 7.5, 8.3, 2.0, 8.0, 2.2, 7.4, 8.3, 8.0, 8.3, 8.1, 7.8, 6.9, 8.3, 7.4, 7.2, 1.1, 7.9, 2.5, 8.0, 8.1, 7.8, 1.0, 7.9, 8.0, 1.2, 8.1, 1.6, 8.3, 1.4, 7.6, 1.4, 8.4, 1.0, 5.3, 6.9, 7.8, 8.2, 1.0, 1.3, 2.2, 1.0, 6.4, 7.1, 8.4, 7.9, 7.7, 8.0, 2.1, 8.6, 8.2, 6.6, 7.7, 7.6, 8.1, 8.0, 7.3, 5.2, 7.9, 8.3, 1.1, 1.2, 8.4, 8.3, 8.3, 0.9, 8.1, 1.0, 7.2, 8.0, 7.8, 2.8, 8.4, 7.6, 7.9, 8.3, 7.2, 7.9, 8.

...

ROC-AUC Score: 0.9999
Average training loss: 0.0014
#
Running Validation...


Validating:: 100%|██████████| 29/29 [00:06<00:00,  4.29it/s]


(907,)
[9.8, 9.8, 9.7, 9.8, 9.7, 9.8, 9.8, 9.8, 9.8, 9.8, 9.2, 0.4, 0.4, 9.7, 9.8, 9.8, 9.8, 9.8, 9.7, 9.8, 9.8, 9.8, 9.7, 9.8, 9.7, 9.8, 9.8, 9.8, 9.8, 9.7, 9.8, 9.8, 9.8, 9.8, 9.8, 9.8, 0.9, 9.8, 9.6, 9.8, 0.3, 9.8, 0.4, 9.8, 9.8, 9.8, 9.6, 9.8, 9.3, 0.4, 9.8, 9.7, 9.5, 0.5, 9.7, 9.8, 0.4, 0.4, 9.8, 9.4, 9.8, 9.6, 0.5, 9.8, 9.7, 9.8, 0.3, 0.4, 9.8, 9.8, 9.8, 9.8, 0.6, 9.7, 9.7, 9.7, 9.8, 9.8, 9.7, 9.8, 9.8, 9.8, 9.7, 9.8, 9.7, 0.6, 9.8, 0.4, 9.8, 0.4, 9.8, 9.8, 9.8, 9.7, 9.8, 9.8, 9.8, 9.8, 9.8, 9.7, 9.8, 0.4, 9.8, 9.8, 9.8, 9.8, 9.8, 9.8, 9.8, 9.8, 0.4, 9.7, 0.6, 0.4, 9.8, 0.6, 9.8, 0.4, 9.8, 0.4, 9.8, 9.8, 0.7, 0.5, 9.8, 9.8, 9.8, 9.8, 9.8, 9.7, 9.8, 9.8, 9.8, 9.6, 9.6, 9.8, 9.7, 0.3, 0.5, 9.8, 0.3, 9.7, 9.7, 9.6, 0.6, 9.8, 9.8, 0.5, 9.8, 9.8, 9.8, 9.8, 9.8, 9.6, 9.8, 0.9, 9.8, 9.8, 9.8, 9.6, 9.8, 0.4, 0.5, 9.7, 9.7, 9.7, 0.3, 0.7, 9.8, 9.8, 9.8, 9.5, 9.8, 9.8, 9.8, 0.4, 9.8, 0.4, 0.4, 9.8, 9.8, 0.5, 9.7, 9.8, 9.8, 0.7, 0.5, 9.8, 9.1, 9.8, 9.7, 9.7, 9.7, 9.8, 9.8, 9.8, 0.5, 9.8, 9.

Testing...: 100%|██████████| 160/160 [00:39<00:00,  4.06it/s]


Training complete!


In [ ]:
!nvidia-smi

In [ ]:
!pip install numba

In [ ]:
from numba import cuda

device = cuda.get_current_device() 
device.reset()
cuda.close()